In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from planner import JourneyPlanner
from collections import defaultdict

In [9]:
paper_conn = [
    ("y", "t", 100000, 110000, "1"),
    ("z", "t", 90000, 120000, "2"),
    ("x", "t", 80000, 130000, "3"),
    ("x", "y", 80000, 90000, "4"),
    ("s", "z", 70000, 80000, "5"),
    ("s", "x", 60000, 70000, "6"),
    ("s", "t", 50000, 140000, "7")
]
timetable = (None, paper_conn, None, defaultdict(lambda: list()))

In [10]:
planner = JourneyPlanner(timetable)

planner.plan_route("s", "t", 40000, 150000, max_connections=3, include_earliest_arrival=True)

JOURNEY 0
At 19:26:40, take the line 2 with arrival at time 09:20:00 in t
JOURNEY 1
At 16:40:00, take the line 6 with arrival at time 19:26:40 in x
After reaching station x, change from line 6 to line 3 (estimated change time of 60 seconds)
At 22:13:20, take the line 3 with arrival at time 12:06:40 in t
JOURNEY 2
At 16:40:00, take the line 6 with arrival at time 19:26:40 in x
After reaching station x, change from line 6 to line 4 (estimated change time of 60 seconds)
At 22:13:20, take the line 4 with arrival at time 01:00:00 in y
After reaching station y, change from line 4 to line 1 (estimated change time of 60 seconds)
At 03:46:40, take the line 1 with arrival at time 06:33:20 in t
